# Sanity check

In [1]:
echo ${RENKU_USERNAME}

ceraolo


In [2]:
hdfs dfs -ls /data/sbb/csv/stop_times/2019/05/

Found 6 items
drwxrwxrwx   - hdfs hdfs          0 2022-05-16 22:27 /data/sbb/csv/stop_times/2019/05/01
drwxrwxrwx   - hdfs hdfs          0 2022-05-16 22:28 /data/sbb/csv/stop_times/2019/05/02
drwxrwxrwx   - hdfs hdfs          0 2022-05-16 22:28 /data/sbb/csv/stop_times/2019/05/08
drwxrwxrwx   - hdfs hdfs          0 2022-05-16 21:45 /data/sbb/csv/stop_times/2019/05/15
drwxrwxrwx   - hdfs hdfs          0 2022-05-16 21:45 /data/sbb/csv/stop_times/2019/05/22
drwxrwxrwx   - hdfs hdfs          0 2022-05-16 21:45 /data/sbb/csv/stop_times/2019/05/29


In [16]:
hdfs dfs -ls /group/five-guys/

Found 7 items
drwxr-xr-x+  - eric hadoop          0 2022-05-24 16:15 /group/five-guys/conn_table
drwxrwxr-x+  - eric hadoop          0 2022-05-24 15:28 /group/five-guys/fridaycsv
drwxrwxr-x+  - hive hadoop          0 2022-05-24 14:45 /group/five-guys/hive
drwxrwxr-x+  - eric hadoop          0 2022-05-24 14:40 /group/five-guys/mondaycsv
drwxrwxr-x+  - eric hadoop          0 2022-05-24 15:27 /group/five-guys/thursdaycsv
drwxrwxr-x+  - eric hadoop          0 2022-05-24 15:23 /group/five-guys/tuesdaycsv
drwxrwxr-x+  - eric hadoop          0 2022-05-24 15:26 /group/five-guys/wednesdaycsv


# Data description
We provide a summary description of the files below. The most relevant files are marked by (+):

**Note:** PK=Primary Key (unique), FK=Foreign Key (refers to a Primary Key in another table)

#### Stops locations

* stops.txt(+):

    - `STOP_ID`: unique identifier (PK) of the stop
    - `STOP_NAME`: long name of the stop
    - `STOP_LAT`: stop latitude (WGS84)
    - `STOP_LON`: stop longitude
    - `LOCATION_TYPE`:
    - `PARENT_STATION`: if the stop is one of many collocated at a same location, such as platforms at a train station

In [3]:
hdfs dfs -cat /data/sbb/csv/allstops/stop_locations.csv | head

"1100006","Zell (Wiesental), Bahnhof","47.7046317515335","7.84777215468897","",""
"1100008","Zell (Wiesental), Wilder Mann","47.7100842702352","7.85964788274668","",""
"1100009","Zell (Wiesental), Grönland","47.7131911044794","7.86290876722849","",""
"1100010","Atzenbach","47.7146175266411","7.8723500608659","",""
"1100011","Mambach, Brücke","47.7282088873189","7.8774704579861","",""
"1100012","Mambach, Mühlschau","47.7340818684375","7.8813871126254","",""
"1100013","Mambach, Silbersau","47.7395192233867","7.88223152899259","",""
"1100014","Fröhnd (Schwarzw), Wühre","47.7543663509316","7.88913059037559","",""
"1100015","Fröhnd (Schwarzw), Unterkastel","47.7605926689054","7.88553732923861","",""
"1100016","Wembach (Baden)","47.7728317637339","7.88772023537933","",""
cat: Unable to write to output stream.


In [10]:
hdfs dfs -cat /data/sbb/csv/allstops/stops.txt | head

cat: `/data/sbb/csv/allstops/stops.txt': No such file or directory


#### Stop Times

* stop_times.txt(+):

    - `TRIP_ID`: identifier (FK) of the trip, unique for the day - e.g. _1.TA.1-100-j19-1.1.H_
    - `ARRIVAL_TIME`: scheduled (local) time of arrival at the stop (same as DEPARTURE_TIME if this is the start of the journey)
    - `DEPARTURE_TIME`: scheduled (local) time of departure at the stop 
    - `STOP_ID`: stop (station) identifier (FK), from stops.txt
    - `STOP_SEQUENCE`: sequence number of the stop on this trip id, starting at 1.
    - `PICKUP_TYPE`:
    - `DROP_OFF_TYPE`:

In [4]:
hdfs dfs -cat /data/sbb/csv/stop_times/2018/01/03/stop_times.txt | head

﻿trip_id,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type
"1.TA.1-88-j18-1.1.R","05:45:00","05:45:00","8572313","1","0","0"
"1.TA.1-88-j18-1.1.R","05:46:00","05:46:00","8572258","2","0","0"
"1.TA.1-88-j18-1.1.R","05:49:00","05:49:00","8580475","3","0","0"
"1.TA.1-88-j18-1.1.R","05:49:00","05:49:00","8572256","4","0","0"
"1.TA.1-88-j18-1.1.R","05:51:00","05:51:00","8582231","5","0","0"
"1.TA.1-88-j18-1.1.R","05:51:00","05:51:00","8582230","6","0","0"
"2.TA.1-88-j18-1.2.R","06:05:00","06:05:00","8582230","1","0","0"
"2.TA.1-88-j18-1.2.R","06:05:00","06:05:00","8582231","2","0","0"
"2.TA.1-88-j18-1.2.R","06:07:00","06:07:00","8582232","3","0","0"
cat: Unable to write to output stream.


### Trips

* trips.txt:

    - `ROUTE_ID`: identifier (FK) for the route. A route is a sequence of stops. It is time independent.
    - `SERVICE_ID`: identifier (FK) of a group of trips in the calendar, and for managing exceptions (e.g. holidays, etc).
    - `TRIP_ID`: is one instance (PK) of a vehicle journey on a given route - the same route can have many trips at regular intervals; a trip may skip some of the route stops.
    - `TRIP_HEADSIGN`: displayed to passengers, most of the time this is the (short) name of the last stop.
    - `TRIP_SHORT_NAME`: internal identifier for the trip_headsign (note TRIP_HEADSIGN and TRIP_SHORT_NAME are only unique for an agency)
    - `DIRECTION_ID`: if the route is bidirectional, this field indicates the direction of the trip on the route.

In [5]:
hdfs dfs -cat /data/sbb/csv/trips/2018/01/03/trips.txt | head

﻿route_id,service_id,trip_id,trip_headsign,trip_short_name,direction_id
"1-9-j18-1","TA+b0001","4.TA.1-9-j18-1.3.H","Lenzburg","21916","0"
"1-ZB1-j18-1","TA+b0001","1.TA.1-ZB1-j18-1.1.R","Bad Zurzach, Bahnhof","101","1"
"1-ZB1-j18-1","TA+b0001","2.TA.1-ZB1-j18-1.1.R","Bad Zurzach, Bahnhof","103","1"
"1-ZB1-j18-1","TA+b0001","3.TA.1-ZB1-j18-1.1.R","Bad Zurzach, Bahnhof","105","1"
"1-ZB1-j18-1","TA+b0001","4.TA.1-ZB1-j18-1.1.R","Bad Zurzach, Bahnhof","107","1"
"1-ZB1-j18-1","TA+b0001","5.TA.1-ZB1-j18-1.1.R","Bad Zurzach, Bahnhof","109","1"
"1-ZB1-j18-1","TA+b0001","6.TA.1-ZB1-j18-1.1.R","Bad Zurzach, Bahnhof","111","1"
"1-ZB1-j18-1","TA+b0001","7.TA.1-ZB1-j18-1.1.R","Bad Zurzach, Bahnhof","113","1"
"1-ZB1-j18-1","TA+b0001","8.TA.1-ZB1-j18-1.1.R","Bad Zurzach, Bahnhof","115","1"
cat: Unable to write to output stream.


### Calendar

* calendar.txt:

    - `SERVICE_ID`: identifier (PK) of a group of trips sharing a same calendar and calendar exception pattern.
    - `MONDAY`..`SUNDAY`: 0 or 1 for each day of the week, indicating occurence of the service on that day.
    - `START_DATE`: start date when weekly service id pattern is valid
    - `END_DATE`: end date after which weekly service id pattern is no longer valid

In [6]:
hdfs dfs -cat /data/sbb/csv/calendar/2018/01/03/calendar.txt | head

﻿service_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date
"TA+b0007","1","1","1","1","1","0","0","20171210","20181208"
"TA+b001s","0","0","0","0","0","0","1","20171210","20181208"
"TA+b001i","0","0","0","0","0","1","0","20171210","20181208"
"TA+b07b6","0","0","0","0","0","1","1","20171210","20181208"
"TA+b0ayl","0","0","0","0","1","0","0","20171210","20181208"
"TA+b0025","1","1","1","1","1","1","0","20171210","20181208"
"TA+b0d47","1","1","1","1","1","0","0","20171210","20181208"
"TA+b0015","0","0","0","0","1","1","0","20171210","20181208"
"TA+b006p","0","0","0","0","0","0","1","20171210","20181208"
cat: Unable to write to output stream.


### Routes

    
* routes.txt:

    - `ROUTE_ID`: identifier for the route (PK)
    - `AGENCY_ID`: identifier of the operator (FK)
    - `ROUTE_SHORT_NAME`: the short name of the route, usually a line number
    - `ROUTE_LONG_NAME`: (empty)
    - `ROUTE_DESC`: _Bus_, _Zub_, _Tram_, etc.
    - `ROUTE_TYPE`:

In [7]:
hdfs dfs -cat /data/sbb/csv/routes/2018/01/03/routes.txt | head

﻿route_id,agency_id,route_short_name,route_long_name,route_desc,route_type
"11-61-j18-1","7031","061","","Bus","700"
"11-62-j18-1","7031","062","","Bus","700"
"24-64-j18-1","801","064","","Bus","700"
"24-65-j18-1","801","065","","Bus","700"
"24-66-j18-1","801","066","","Bus","700"
"24-67-j18-1","801","067","","Bus","700"
"11-83-j18-1","801","083","","Bus","700"
"1-1-A-j18-1","11","1","","S-Bahn","400"
"1-1-B-j18-1","11","1","","S-Bahn","400"
cat: Unable to write to output stream.


### Other

The other files are:

* _calendar-dates.txt_ contains exceptions to the weekly patterns expressed in _calendar.txt_.
* _agency.txt_ has the details of the operators
* _transfers.txt_ contains the transfer times between stops or platforms.

In [8]:
hdfs dfs -cat /data/sbb/csv/transfers/2018/01/03/transfers.txt | head

﻿from_stop_id,to_stop_id,transfer_type,min_transfer_time
"8500309:0:2","8500309:0:4","2","180"
"8500309:0:2","8500309:0:5","2","180"
"8500309:0:2","8500309:0:3","2","180"
"8500309:0:2","8500309:0:1","2","180"
"8500309:0:2","8500309:0:5B","2","180"
"8500305:0:3","8500305:0:1","2","180"
"8500305:0:3","8500305:0:2","2","180"
"8500304:0:1","8500304:0:2","2","120"
"8500320:0:4","8500320:0:2","2","120"
cat: Unable to write to output stream.
